In [1]:
import pandas as pd
import numpy as np

In [2]:
import seaborn as sb

In [4]:
sb.get_dataset_names()

/Users/mattwilliams/anaconda3/envs/pfds/lib/python3.8/site-packages/seaborn/utils.py:384: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 384 of the file /Users/mattwilliams/anaconda3/envs/pfds/lib/python3.8/site-packages/seaborn/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  gh_list = BeautifulSoup(http)


['anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'exercise',
 'flights',
 'fmri',
 'gammas',
 'iris',
 'mpg',
 'planets',
 'tips',
 'titanic']

In [24]:
flights = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/flights.csv')

In [25]:
flights.head()

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


In [26]:
flights_indexed = flights.set_index(['year', 'month'])
flights_indexed.head()

passengers
year month               
1949 January          112
     February         118
     March            132
     April            129
     May              121

In [27]:
flights_indexed.loc[1949]

,passengers
month,
January,112
February,118
March,132
April,129
May,121
June,135
July,148
August,148
September,136


In [28]:
flights_indexed.loc[1949:1950]

passengers
year month                
1949 January           112
     February          118
     March             132
     April             129
     May               121
     June              135
     July              148
     August            148
     September         136
     October           119
     November          104
     December          118
1950 January           115
     February          126
     March             141
     April             135
     May               125
     June              149
     July              170
     August            170
     September         158
     October           133
     November          114
     December          140

In [29]:
flights_indexed.loc[1949, 'January']

passengers    112
Name: (1949, January), dtype: int64

In [30]:
flights_indexed.loc[1949].loc['January':'March']

,passengers
month,
January,112
February,118
March,132


If we 'unstack' the multi-indexed dataframe, we will get a multi-indexed column construct. However, this will allow us to do things like sum all the flights for a given year.

In [31]:
flights_unstacked = flights_indexed.unstack()
flights_unstacked

passengers                                                        \
month      April August December February January July June March  May   
year                                                                     
1949         129    148      118      118     112  148  135   132  121   
1950         135    170      140      126     115  170  149   141  125   
1951         163    199      166      150     145  199  178   178  172   
1952         181    242      194      180     171  230  218   193  183   
1953         235    272      201      196     196  264  243   236  229   
1954         227    293      229      188     204  302  264   235  234   
1955         269    347      278      233     242  364  315   267  270   
1956         313    405      306      277     284  413  374   317  318   
1957         348    467      336      301     315  465  422   356  355   
1958         348    505      337      318     340  491  435   362  363   
1959         396    559      405      342     360  548  472   406  420   
1960         461    606      432      391     417  622  535   419  472   

                                  
month November October September  
year                              
1949       104     119       136  
1950       114     133       158  
1951       146     162       184  
1952       172     191       209  
1953       180     211       237  
1954       203     229       259  
1955       237     274       312  
1956       271     306       355  
1957       305     347       404  
1958       310     359       404  
1959       362     407       463  
1960       390     461       508

In [32]:
flights_unstacked.sum(axis=1)

year
1949    1520
1950    1676
1951    2042
1952    2364
1953    2700
1954    2867
1955    3408
1956    3939
1957    4421
1958    4572
1959    5140
1960    5714
dtype: int64

These results can be assigned to a new column, taking note that the columns are now multi-level.

In [33]:
flights_unstacked['passengers', 'total'] = flights_unstacked.sum(axis=1)

In [34]:
flights_unstacked

passengers                                                        \
month      April August December February January July June March  May   
year                                                                     
1949         129    148      118      118     112  148  135   132  121   
1950         135    170      140      126     115  170  149   141  125   
1951         163    199      166      150     145  199  178   178  172   
1952         181    242      194      180     171  230  218   193  183   
1953         235    272      201      196     196  264  243   236  229   
1954         227    293      229      188     204  302  264   235  234   
1955         269    347      278      233     242  364  315   267  270   
1956         313    405      306      277     284  413  374   317  318   
1957         348    467      336      301     315  465  422   356  355   
1958         348    505      337      318     340  491  435   362  363   
1959         396    559      405      342     360  548  472   406  420   
1960         461    606      432      391     417  622  535   419  472   

                                        
month November October September total  
year                                    
1949       104     119       136  1520  
1950       114     133       158  1676  
1951       146     162       184  2042  
1952       172     191       209  2364  
1953       180     211       237  2700  
1954       203     229       259  2867  
1955       237     274       312  3408  
1956       271     306       355  3939  
1957       305     347       404  4421  
1958       310     359       404  4572  
1959       362     407       463  5140  
1960       390     461       508  5714

In [35]:
flights_restacked = flights_unstacked.stack()
flights_restacked

passengers
year month                
1949 April             129
     August            148
     December          118
     February          118
     January           112
...                    ...
1960 May               472
     November          390
     October           461
     September         508
     total            5714

[156 rows x 1 columns]

Now, the total is included in the second level index. If we wanted to see only the total passengers for each year:

In [36]:
flights_restacked.loc[pd.IndexSlice[:, 'total'], 'passengers']

year  month
1949  total    1520
1950  total    1676
1951  total    2042
1952  total    2364
1953  total    2700
1954  total    2867
1955  total    3408
1956  total    3939
1957  total    4421
1958  total    4572
1959  total    5140
1960  total    5714
Name: passengers, dtype: int64

In [37]:
flights_restacked[flights_restacked['passengers'] > 120]

passengers
year month                
1949 April             129
     August            148
     July              148
     June              135
     March             132
...                    ...
1960 May               472
     November          390
     October           461
     September         508
     total            5714

[149 rows x 1 columns]